Some code inherited from christianversloot (github)'s machine learning articles and Deep Learning in Python (Chollet)

Preprocessing:

In [ ]:
# importing the sys module
import sys
sys.path.append('C:/Users/nsshi/github/ThesisFinal/src/modules')  
import lib
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf

# File data
save_dir = '/modelsOut/'
emg_filename = r'Data\EMG_Mar_19_09_34_15.txt'
q_filename = r'Data\Q_Mar_19_09_33_45.txt'

# Preprocessing
emg, emgTimes, q, qTimes = lib.import_files(emg_filename, q_filename, old=True)
qF, emgF, tF, timestampsF = lib.unify_timeseries_high(emg, emgTimes, q, qTimes)
emgModel = lib.normalize_emg_rolling(emgF)
qModel = lib.finger_nodes(qF, finger = 'indextip')


In [ ]:
sequence_length = 25
trn, val, tst, input_shape, output_shape, n_features = lib.create_feature_dataset(tF, emgModel, qModel, sequence_length = sequence_length, stride_features=10, full = False)

Naive Method:

In [ ]:
naive_rmse_val, naive_rmse_tst = lib.naive_method(qModel)
print("Naive Validation Error:", naive_rmse_val)
print("Naive Test Error:", naive_rmse_tst)

Basic Dense Model:

In [ ]:
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))

# Dense layers
x = layers.Flatten()(inputs)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dense(8, activation="relu")(x)
x = layers.Dense(4, activation="relu")(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_basic_dense.keras",
 save_best_only=True)
] 
model.compile(optimizer = "rmsprop", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs = 10,
 validation_data = val,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_basic_dense.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

In [ ]:
# More complex model
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))
x = layers.Flatten()(inputs)
x = layers.Dense(1200, activation="relu")(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dense(10, activation="relu")(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_heavy_dense.keras",
 save_best_only=True)
] 
model.compile(optimizer = "rmsprop", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs = 5,
 validation_data = val,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_heavy_dense.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

In [ ]:
# Even more complex model
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))
x = layers.Flatten()(inputs)
x = layers.Dense(1200, activation="relu")(x)
x = layers.Dense(600, activation="relu")(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dense(10, activation="relu")(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_heaviest_dense.keras",
 save_best_only=True)
] 
model.compile(optimizer = "rmsprop", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs = 5,
 validation_data = val,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_heaviest_dense.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

Conv1D Method:

In [ ]:
# Light Model
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))
x = layers.Conv1D(8, 16, activation="relu", padding="causal")(inputs)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(8, 8, activation="relu", padding="causal")(x)
# x = layers.MaxPooling1D(2)(x)
# x = layers.Conv1D(8, 4, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_basic_conv1D.keras",
 save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=20,
 validation_data=val,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_basic_conv1D.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

LSTM:

In [ ]:
# Basic LSTM
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))
x = layers.LSTM(32)(inputs)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_basic_LSTM.keras",
 save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=20,
 validation_data=val,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_basic_LSTM.keras") 
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

GRU:

In [ ]:
# Basic Stacked GRU
do = 0.8
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))
x = layers.GRU(4, recurrent_dropout=do, return_sequences=True)(inputs)
x = layers.GRU(4, recurrent_dropout=do, return_sequences=True)(x)
x = layers.GRU(4, recurrent_dropout=do)(x)
x = layers.Dropout(do)(x)
x = layers.Dense(4)(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_basic_stacked_GRU.keras",
 save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=10,
 validation_data=val,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_basic_stacked_GRU.keras")

In [ ]:
lib.model_summary(model, history, tst)

Models from Literature

Nguyen (2021): (Classification?)

In [ ]:
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))

# Convolutional layer
x = layers.Conv1D(64, 3, activation="relu", padding="causal")(inputs)
x = layers.BatchNormalization()(x)

# Recurrent layers
x = layers.GRU(512, dropout = 0.5, recurrent_dropout = 0.6, return_sequences = True)(x)
x = layers.GRU(512, dropout = 0.5, recurrent_dropout = 0.6)(x)

# Output layer
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_Nguyen_GRU.keras",
 save_best_only=True)
]
opt = tf.keras.optimizers.Adam(learning_rate=0.0001,
    beta_1=0.99,
    beta_2=0.999,
    epsilon=1e-05,
)
model.compile(optimizer=opt, loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=10,
 validation_data=val,
 batch_size=64,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_Nguyen_GRU.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

More brilliance from Nguyen:

Periodgram is used as input in literature, but in this case is relatively sparse of information.

In [ ]:
import scipy.signal as signal
import matplotlib.pyplot as plt

print(emgF.shape)

# Raw signals
lib.plot_emg(tF, emgF)
fs = 1/np.mean(np.diff(tF))

# Periodgram
f, Pxx_den = signal.periodogram(emgF[:,11], fs)
plt.semilogy(f, Pxx_den)
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

# Spectrogram
f, t, Sxx = signal.spectrogram(emgF[:,10], fs, axis=0)
print(Sxx.shape)
# plt.pcolormesh(t, f, Sxx[:,0,:])
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()


In [ ]:
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))

# Input Layer
x = layers.Conv1D(32, 3, padding="causal")(inputs)
x = layers.BatchNormalization()(x)
x0 = layers.ReLU()(x)

# Residual layer helper function
def res(x0):
    x1 = layers.AveragePooling1D(pool_size=2, strides=2)(x0)
    x = layers.Conv1D(32, 3, strides = 2, padding="causal")(x0)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv1D(32, 3, padding="causal")(x)
    x = layers.Add()([x, x1])
    x = layers.BatchNormalization()(x)
    x0 = layers.ReLU()(x)

    x1 = layers.AveragePooling1D(pool_size=2, strides=2)(x0)
    x = layers.Conv1D(32, 3, strides = 1, padding="causal")(x0)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv1D(32, 3, padding="causal")(x)
    x = layers.Add()([x, x0])
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x0 = layers.Dropout(0.1)(x)
    return x0

# Residual layers (Nguyen uses five, but has larger input dimensions)
x0 = res(x0)
x0 = res(x0)

# Recurrent layers
x = layers.GRU(64, dropout = 0.2, recurrent_dropout = 0, return_sequences = True)(x0)
x1 = layers.GRU(64, dropout = 0.2, recurrent_dropout = 0)(x)

# Attention layers
x = layers.Dense(64, activation="relu")(x1)
x = layers.Dense(1)(x)
x = layers.Multiply()([x, x1])
x = layers.Softmax()(x)

# Output layer
outputs = layers.Dense(output_shape, activation="sigmoid")(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_Nguyen_Stacked.keras",
 save_best_only=True)
]
opt = tf.keras.optimizers.Adam(learning_rate=0.01,
    beta_1=0.99,
    beta_2=0.999,
    epsilon=1e-05,
)
model.compile(optimizer=opt, loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=10,
 validation_data=val,
 batch_size=64,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_Nguyen_Stacked.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

Luu (2021): Regression Convolutional Neural Network

In [ ]:
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))

# Convolutional Layers
# Layer 0
x = layers.Conv1D(32, 3, strides = 1, padding="causal")(inputs)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

# Layer 1
x = layers.Conv1D(64, 3, strides = 2, padding="causal")(x)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

# Layer 2
x = layers.Conv1D(64, 3, strides = 1, padding="causal")(x)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

# Layer 3
x = layers.Conv1D(64, 3, strides = 1, padding="causal")(x)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

# Layer 4
x = layers.Conv1D(50, 3, strides = 1, padding="causal")(x)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)

# Output
x = layers.Flatten()(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_Luu_Conv1D.keras",
 save_best_only=True)
]
opt = tf.keras.optimizers.Adam(learning_rate=0.005,
    beta_1=0.99,
    beta_2=0.999,
    epsilon=1e-05,
)
model.compile(optimizer='adam', loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=5,
 validation_data=val,
 batch_size=64,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_Luu_Conv1D.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

Luu (2021): Regression Recurrent Neural Network

In [ ]:
inputs = keras.Input(shape = (n_features, emgModel.shape[-1]))

# Convolutional Layers
# Layer 0
x = layers.Conv1D(32, 3, strides = 1, padding="causal")(inputs)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

# Layer 1
x = layers.Conv1D(64, 3, strides = 2, padding="causal")(x)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

# Layer 2
x = layers.Conv1D(64, 3, strides = 1, padding="causal")(x)
x = layers.ReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

x = layers.LSTM(64, dropout = 0.2, recurrent_dropout = 0, return_sequences = True)(x)
outputs = layers.LSTM(output_shape, dropout = 0.2, recurrent_dropout = 0)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_Luu_RNN.keras",
 save_best_only=True)
]
opt = tf.keras.optimizers.Adam(learning_rate=0.005,
    beta_1=0.99,
    beta_2=0.999,
    epsilon=1e-05,
)
model.compile(optimizer=opt, loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=10,
 validation_data=val,
 batch_size=38,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_Luu_RNN.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

Lin (2022): (Classification?)

In [ ]:
inputs = keras.Input(shape = (n_features, emgModel.shape[-1], 1))

# Convolutional Layers
# Layer 0
x = layers.Conv2D(32, (3,int(sequence_length/10)), padding="valid")(inputs)
x = layers.BatchNormalization()(x)
x = layers.PReLU()(x)
x = layers.Dropout(0.5)(x)

x = layers.MaxPooling2D(pool_size=(1,3))(x)
x = layers.BatchNormalization()(x)
x = layers.PReLU()(x)
x = layers.Dropout(0.5)(x)

x = layers.Conv2D(32, (3,int(sequence_length/10)), padding="valid")(inputs)
x = layers.BatchNormalization()(x)
x = layers.PReLU()(x)
x = layers.Dropout(0.5)(x)

x = layers.MaxPooling2D(pool_size=(1,3))(x)
x = layers.BatchNormalization()(x)
x = layers.PReLU()(x)
x = layers.Dropout(0.5)(x)

x = layers.Flatten()(x)
x = layers.Dense(500)(x)
x = layers.ReLU()(x)
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_Lin_Conv2D.keras",
 save_best_only=True)
]
opt = tf.keras.optimizers.Adam(learning_rate=0.005,
    beta_1=0.99,
    beta_2=0.999,
    epsilon=1e-05,
)
model.compile(optimizer='adam', loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=10,
 validation_data=val,
 batch_size=38,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_Lin_Conv2D.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)

Hajin (2022): CNNs

In [ ]:
inputs = keras.Input(shape=(sequence_length, input_shape,1))

# Convolutional Stack 0
# Convolutional Layer 0
x = layers.Conv2D(128, (3,3), padding="same")(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

# Convolutional Layer 1
x = layers.Conv2D(128, (3,3), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)

# Convolutional Layer 2
x = layers.Conv2D(256, (3,3), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

# Convolutional Layer 3
x = layers.Conv2D(256, (3,3), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)

# Convolutional Layer 4
x = layers.Conv2D(256, (3,3), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Dropout(0.5)(x)

# Output Layer
x = layers.Flatten()(x)

# Convolutional Stack 1
# Convolutional Layer 0
x1 = layers.Conv2D(128, (7,7), padding="same")(inputs)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ReLU()(x1)

# Convolutional Layer 1
x1 = layers.Conv2D(128, (7,7), padding="same")(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ReLU()(x1)
x1 = layers.MaxPooling2D(pool_size=(2,2))(x1)

# Convolutional Layer 2
x1 = layers.Conv2D(256, (7,7), padding="same")(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ReLU()(x1)

# Convolutional Layer 3
x1 = layers.Conv2D(256, (7,7), padding="same")(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ReLU()(x1)
x1 = layers.MaxPooling2D(pool_size=(2,2))(x1)

# Convolutional Layer 4
x1 = layers.Conv2D(256, (7,7), padding="same")(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ReLU()(x1)
x1 = layers.MaxPooling2D(pool_size=(2,2))(x1)
x1 = layers.Dropout(0.5)(x1)

# Output layer
x1 = layers.Flatten()(x1)

# Aggregation layer
x = layers.Concatenate()([x, x1])
x = layers.Dense(100)(x)
x = layers.ReLU()(x)
x = layers.Dense(50)(x)
x = layers.ReLU()(x)

outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "27_Hajin_Parallel_Stacked.keras",
 save_best_only=True)
]
opt = tf.keras.optimizers.Adam(learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-05,
)
model.compile(optimizer='adam', loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=15,
 validation_data=val,
 batch_size=128,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "27_Hajin_Parallel_Stacked.keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")

In [ ]:
lib.model_summary(model, history, tst)